In [0]:
## PySpark Jupyter Notebook with Python 3 on Azure Databricks
## Extract the top most frequent used words in reviews for each property listing

In [0]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [0]:
# define schema for reviews.csv.gz
reviews_schema = StructType([
    StructField('location', StringType(), True),
    StructField('listing_id', StringType(), True),
    #StructField('id',         IntegerType(),True),
    StructField('date',       StringType(),True),
    #StructField('reviewer_id', IntegerType(),True),
    StructField('reviewer_name', StringType(),True),
    StructField('comments', StringType(), True)])

In [0]:
# Read reviews parquet file into data frame reviews_df
#cleandata = 'c:/sb/strfacts/cleandata'
cleandata = '/mnt/blob1/cleandata'
reviews_df = spark.read.format("parquet").schema(reviews_schema).load(cleandata)

In [0]:
reviews_df.show(5)

+----------+----------+----------------+--------------------+--------+
listing_id| date| reviewer_name| comments|location|
+----------+----------+----------------+--------------------+--------+
 19965|2012-01-10| Derick| We loved our sta...| rome|
 12398|2010-10-18| Samuel| "Gea was a wonde...| rome|
 19965|2012-06-12| Hannah| The apartment wa...| rome|
 12398|2010-10-24| Debra| "We really loved...| rome|
 12398|2010-11-03|Tahnee & Michael| "Great apartment...| rome|
+----------+----------+----------------+--------------------+--------+
only showing top 5 rows

In [0]:
# group by list_id, concat all comments
reviews_df.createOrReplaceTempView("reviews")
query = """
    SELECT listing_id, CONCAT_WS(' ', collect_list(comments)) as all_comments, first(location) as area
    FROM reviews
    GROUP BY listing_id
    """
tmpdf = spark.sql(query)
tmpdf.show(2)

+----------+--------------------+-----------+
listing_id| all_comments| area|
+----------+--------------------+-----------+
 10001394| Great place to s...|mexico-city|
 10007435| "Sam's place is ...| hong-kong|
+----------+--------------------+-----------+
only showing top 2 rows

In [0]:
# regex remove all nonalphanumeric characters, split by " " and explode words
df1 = tmpdf.select(tmpdf.listing_id,tmpdf.area, F.explode(F.split(F.regexp_replace(tmpdf.all_comments,"[^a-zA-Z0-9 -]","")," ")).alias("words"))

In [0]:
df1.show(10)

+----------+-----------+--------+
listing_id| area| words|
+----------+-----------+--------+
 10001394|mexico-city| |
 10001394|mexico-city| Great|
 10001394|mexico-city| place|
 10001394|mexico-city| to|
 10001394|mexico-city| stay|
 10001394|mexico-city| This|
 10001394|mexico-city| couple|
 10001394|mexico-city| was|
 10001394|mexico-city| very|
 10001394|mexico-city|inviting|
+----------+-----------+--------+
only showing top 10 rows

In [0]:
# Find the top 5 most frequent used words in the comments
df2 = df1.groupBy(df1.listing_id, df1.area, df1.words)\
      .agg(F.count(df1.words).alias("cnt"))

In [0]:
df2.show(5)

+----------+-----------+-----+---+
listing_id| area|words|cnt|
+----------+-----------+-----+---+
 10001394|mexico-city| |284|
 10001394|mexico-city|Great| 4|
 10001394|mexico-city|place| 6|
 10001394|mexico-city| to| 25|
 10001394|mexico-city| stay| 10|
+----------+-----------+-----+---+
only showing top 5 rows

In [0]:
windowSpec  = Window.partitionBy(df2.listing_id).orderBy(df2.cnt)
df3 = df2.withColumn("row_number",F.row_number().over(windowSpec))
df3.show(3)

+----------+-----------+--------+---+----------+
listing_id| area| words|cnt|row_number|
+----------+-----------+--------+---+----------+
 10001394|mexico-city| This| 1| 1|
 10001394|mexico-city| couple| 1| 2|
 10001394|mexico-city|inviting| 1| 3|
+----------+-----------+--------+---+----------+
only showing top 3 rows

In [0]:
df4 = df3.select("*").where(df3.row_number<=5)\
      .select(df3.listing_id, df3.area, df3.words)\
      .groupBy(df3.listing_id, df3.area)\
      .agg(F.collect_list(df3.words)\
      .alias("frequentw"))\
      .select(df3.listing_id, df3.area, F.array_join("frequentw", ",")\
      .alias("most_frequent_used_words"))

In [0]:
df4.show(10)

+----------+--------------+------------------------+
listing_id| area|most_frequent_used_words|
+----------+--------------+------------------------+
 10001394| mexico-city| This,couple,invit...|
 10007435| hong-kong| Sams,place,very,c...|
 10012138|broward-county| outdated,few,crac...|
 10015087| toronto| Very,much,appreci...|
 10023519| singapore| enjoyed,Very,staf...|
 1003202| montreal| nice,who,live,are...|
 10034090| new-york-city| Thank,so,much,for...|
 10050466| buenos-aires| apartment,really,...|
 10062514| barcelona| canceled,this,res...|
 10068318| san-diego| needs,only,3,mile...|
+----------+--------------+------------------------+
only showing top 10 rows

In [0]:
#analyticsdata = 'c:/sb/strfacts/analyticsdata'
analyticsdata = '/mnt/blob1/analyticsdata'
df4.write.partitionBy("area").mode("append").parquet(analyticsdata)